In [62]:
#Imports
import json
import pandas as pd
from tqdm import tqdm as loader
import numpy as np
import pymongo
from pymongo import MongoClient
import csv
import sys, getopt, pprint
print(sys.getrecursionlimit())
sys.setrecursionlimit(300000000)
print(sys.getrecursionlimit())

300000000
300000000


In [63]:
!pip3 install tqdm
!pip install pymongo

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [64]:
myclient = pymongo.MongoClient('localhost', 27017)
database = myclient['Assignment2Part01']

In [65]:
#Helper functions
def csv_to_json(filename,col_names):
    df = pd.read_csv(filename ,encoding = 'ISO-8859-1',names=col_names, header=None)
    return df
def filter_comments(post_id):
    try:
        filtered_comments = comments.loc[comments['post_ID'] == post_id]
        filtered_comments = filtered_comments.filter(items=['comment_ID', 'comment_content', 'vote', 'comment_author','created_date'])
        return list(filtered_comments.to_dict('records'))
    except:
        return list()
def filter_inlinks(post_id):
    try:
        filtered_inlinks = inlinks.loc[inlinks['post_ID'] == post_id]
        filtered_inlinks = filtered_inlinks.filter(items=['inlink_ID', 'title', 'author', 'date', 'URL'])
        return list(filtered_inlinks.to_dict('records'))
    except:
        return list()
def df_to_dict(dataframe):
    return dataframe.to_dict('records')
def create_collection(df,collectionName):
    collection = database[collectionName]
    return collection.insert_many(df_to_dict(df))


In [66]:
#Column 
authors_col = ['author_ID','author_name', 'MEIBI','MEIBIX','avg_words_perpost','avg_words_perpost_nostopwords']
comments_col = ['comment_ID','post_ID','comment_content','comment_author','created_date','vote']
inlinks_col = ['inlink_ID','post_ID','title','author','date','URL']
blogpost_col = ['post_ID','title','author_Name','author_id','num_comments','post_content','URL','date','num_retrieved_inlinks','num_retrieved_comments','post_length','post_length_no_stopwords', 'avd_word_length','avg_word_length_no_stopwords','MEIBI','MEBIX']

#Read
authors = csv_to_json('authors.csv', authors_col)
posts = csv_to_json('posts.csv', blogpost_col)
comments = csv_to_json('comments.csv', comments_col)
inlinks = csv_to_json('inlinks.csv', inlinks_col)

In [67]:
posts = posts.drop('num_comments', axis =1)
posts

,post_ID,title,author_Name,author_id,post_content,URL,date,num_retrieved_inlinks,num_retrieved_comments,post_length,post_length_no_stopwords,avd_word_length,avg_word_length_no_stopwords,MEIBI,MEBIX
0,1,We Just Tested Twitter&#8217;s @anywhere Platf...,Jason Kincaid,1,"During his keynote at SXSW last month, Twitter...",http://techcrunch.com/2010/04/01/we-just-teste...,2010-04-01,0,14,314,223,4.130293,5.686099,0.00000,0.0
1,2,Facebook To Launch &#8216;Relationships&#8217;...,Jason Kincaid,1,Later this month Facebook is holding its f8 co...,http://techcrunch.com/2010/04/01/facebook-to-l...,2010-04-01,0,30,717,488,4.138177,5.952869,0.00000,0.0
2,3,NYTimes Request Correction/Removal Of Our Post...,Michael Arrington,2,A NYTimes communications person emailed to ask...,http://techcrunch.com/2010/04/01/nytimes-reque...,2010-04-01,4,38,249,168,3.572614,5.125000,624.00000,0.0
3,4,Google Shows How HTML5 Can Run Quake In The Br...,Erick Schonfeld,3,A lot of attention lately has been put on the ...,http://techcrunch.com/2010/04/01/google-html5-...,2010-04-01,10,26,256,172,3.628571,5.168605,1080.00000,0.0
4,5,The iPad: Apple&#8217;s Trojan Horse,John Biggs,4,Something struck me about Apple&#8217;s handli...,http://www.crunchgear.com/2010/04/01/the-ipad-...,2010-04-01,1,49,503,348,3.773279,5.356322,200.00000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19459,19460,Google Reviews Profile,Michael Arrington,2,Company: Google (Reviews)What is it?There&#821...,http://techcrunch.com/2005/06/12/google-reviews/,2005-06-12,0,3,209,141,3.737864,5.460993,0.00000,0.0
19460,19461,BackPack Profile,Michael Arrington,2,Company: BackPackWhat is it?BackPack launched ...,http://techcrunch.com/2005/06/12/backpack-show...,2005-06-12,1,6,572,412,4.035842,5.466019,0.01595,0.0
19461,19462,Plazes Profile,Michael Arrington,2,Company: PlazesWhat is it?Plazes made an annou...,http://techcrunch.com/2005/06/11/plazes-gets-t...,2005-06-11,10,12,292,217,4.313380,5.645161,0.29613,0.0
19462,19463,FeedLounge Profile,Michael Arrington,2,Company: FeedLoungeWhat is it?The FeedLounge w...,http://techcrunch.com/2005/06/11/get-in-line-t...,2005-06-11,0,9,211,159,4.691176,6.018868,0.00000,0.0


In [68]:
#Get shape
print("Comments shape: {}".format(comments.shape))
print("Authors shape: {}".format(authors.shape))
print("Posts shape: {}".format(posts.shape))
print("Inlinks shape: {}".format(inlinks.shape))

Comments shape: (746561, 6)
Authors shape: (107, 6)
Posts shape: (19464, 15)
Inlinks shape: (193808, 6)


In [69]:
comments

,comment_ID,post_ID,comment_content,comment_author,created_date,vote
0,1,1,Seemed to work fine.,BJ Cook,2010-04-01,1
1,2,1,I tried this the first second you published it...,Blitz Surfer,2010-04-01,1
2,3,1,A lot of twitter apps (including mine) can mak...,Nischal Shetty,2010-04-01,1
3,4,1,"Worked a minute ago, now it&#8217;s pulled",Jonah Grant,2010-04-01,1
4,5,1,Yep. At 10:33pm pst it is down. I get an &#822...,Joshua Guffey,2010-04-01,1
...,...,...,...,...,...,...
746556,746557,5765,This sounds like a non-story. One URL shorten...,Sanjay Sharma,2009-07-20,1
746557,746558,5765,This is a non-story. He knew about it and men...,Amy,2009-07-20,1
746558,746559,5765,Man I feel kind of dumb. I didn&#8217;t even n...,Wallpapers,2009-07-20,1
746559,746560,5765,I don&#8217;t think its that big a deal and mo...,Daniel Licht,2009-07-20,1


In [70]:
inlinks.columns

Index(['inlink_ID', 'post_ID', 'title', 'author', 'date', 'URL'], dtype='object')

In [71]:
post_id = 1 
filtered_comments = comments.loc[comments['post_ID'] == post_id]
filtered_comments = filtered_comments.filter(items=['comment_ID', 'comment_content', 'vote', 'created_date'])
json_ = filtered_comments.to_dict('records')

In [72]:
print(json_)

[{'comment_ID': 1, 'comment_content': 'Seemed to work fine.', 'vote': 1, 'created_date': '2010-04-01'}, {'comment_ID': 2, 'comment_content': 'I tried this the first second you published it,it didn&#8217;t show the @anywhere, it said via Egg Boiling!', 'vote': 1, 'created_date': '2010-04-01'}, {'comment_ID': 3, 'comment_content': 'A lot of twitter apps (including mine) can make use of hovercard to display user info. I would use it as soon as it becomes available!', 'vote': 1, 'created_date': '2010-04-01'}, {'comment_ID': 4, 'comment_content': 'Worked a minute ago, now it&#8217;s pulled', 'vote': 1, 'created_date': '2010-04-01'}, {'comment_ID': 5, 'comment_content': 'Yep. At 10:33pm pst it is down. I get an &#8220;index of&#8221; page. 20 minutes is a pretty good response time on their part. Only wish I had been faster. Thanks for the screenshots!', 'vote': 1, 'created_date': '2010-04-01'}, {'comment_ID': 6, 'comment_content': 'Now it&#8217;s down!I love &#8220;view source&#8221; option 

In [73]:
posts["comments"] = np.nan
posts["comments"] = posts["comments"].astype('object')
posts["inlinks"] = np.nan
posts["inlinks"] = posts["inlinks"].astype('object')

In [74]:
posts.head()

,post_ID,title,author_Name,author_id,post_content,URL,date,num_retrieved_inlinks,num_retrieved_comments,post_length,post_length_no_stopwords,avd_word_length,avg_word_length_no_stopwords,MEIBI,MEBIX,comments,inlinks
0,1,We Just Tested Twitter&#8217;s @anywhere Platf...,Jason Kincaid,1,"During his keynote at SXSW last month, Twitter...",http://techcrunch.com/2010/04/01/we-just-teste...,2010-04-01,0,14,314,223,4.130293,5.686099,0.0,0.0,NaN,NaN
1,2,Facebook To Launch &#8216;Relationships&#8217;...,Jason Kincaid,1,Later this month Facebook is holding its f8 co...,http://techcrunch.com/2010/04/01/facebook-to-l...,2010-04-01,0,30,717,488,4.138177,5.952869,0.0,0.0,NaN,NaN
2,3,NYTimes Request Correction/Removal Of Our Post...,Michael Arrington,2,A NYTimes communications person emailed to ask...,http://techcrunch.com/2010/04/01/nytimes-reque...,2010-04-01,4,38,249,168,3.572614,5.125000,624.0,0.0,NaN,NaN
3,4,Google Shows How HTML5 Can Run Quake In The Br...,Erick Schonfeld,3,A lot of attention lately has been put on the ...,http://techcrunch.com/2010/04/01/google-html5-...,2010-04-01,10,26,256,172,3.628571,5.168605,1080.0,0.0,NaN,NaN
4,5,The iPad: Apple&#8217;s Trojan Horse,John Biggs,4,Something struck me about Apple&#8217;s handli...,http://www.crunchgear.com/2010/04/01/the-ipad-...,2010-04-01,1,49,503,348,3.773279,5.356322,200.0,0.0,NaN,NaN


In [75]:
for idx, post in enumerate(loader(posts.iterrows())):
    post_id = post[1]['post_ID']
    posts.at[idx, 'comments'] =  filter_comments(post_id)
    posts.at[idx, 'inlinks'] = filter_inlinks(post_id)
    

19464it [01:18, 247.89it/s]


In [76]:
posts

,post_ID,title,author_Name,author_id,post_content,URL,date,num_retrieved_inlinks,num_retrieved_comments,post_length,post_length_no_stopwords,avd_word_length,avg_word_length_no_stopwords,MEIBI,MEBIX,comments,inlinks
0,1,We Just Tested Twitter&#8217;s @anywhere Platf...,Jason Kincaid,1,"During his keynote at SXSW last month, Twitter...",http://techcrunch.com/2010/04/01/we-just-teste...,2010-04-01,0,14,314,223,4.130293,5.686099,0.00000,0.0,"[{'comment_ID': 1, 'comment_content': 'Seemed ...",[]
1,2,Facebook To Launch &#8216;Relationships&#8217;...,Jason Kincaid,1,Later this month Facebook is holding its f8 co...,http://techcrunch.com/2010/04/01/facebook-to-l...,2010-04-01,0,30,717,488,4.138177,5.952869,0.00000,0.0,"[{'comment_ID': 15, 'comment_content': 'Is apr...",[]
2,3,NYTimes Request Correction/Removal Of Our Post...,Michael Arrington,2,A NYTimes communications person emailed to ask...,http://techcrunch.com/2010/04/01/nytimes-reque...,2010-04-01,4,38,249,168,3.572614,5.125000,624.00000,0.0,"[{'comment_ID': 258854, 'comment_content': 'lm...","[{'inlink_ID': 513699, 'title': 'Moving into t..."
3,4,Google Shows How HTML5 Can Run Quake In The Br...,Erick Schonfeld,3,A lot of attention lately has been put on the ...,http://techcrunch.com/2010/04/01/google-html5-...,2010-04-01,10,26,256,172,3.628571,5.168605,1080.00000,0.0,"[{'comment_ID': 511943, 'comment_content': 'ty...","[{'inlink_ID': 513689, 'title': 'Quake II func..."
4,5,The iPad: Apple&#8217;s Trojan Horse,John Biggs,4,Something struck me about Apple&#8217;s handli...,http://www.crunchgear.com/2010/04/01/the-ipad-...,2010-04-01,1,49,503,348,3.773279,5.356322,200.00000,0.0,"[{'comment_ID': 588817, 'comment_content': 'yo...","[{'inlink_ID': 513688, 'title': 'Budget Nomad:..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19459,19460,Google Reviews Profile,Michael Arrington,2,Company: Google (Reviews)What is it?There&#821...,http://techcrunch.com/2005/06/12/google-reviews/,2005-06-12,0,3,209,141,3.737864,5.460993,0.00000,0.0,"[{'comment_ID': 570372, 'comment_content': 'Re...",[]
19460,19461,BackPack Profile,Michael Arrington,2,Company: BackPackWhat is it?BackPack launched ...,http://techcrunch.com/2005/06/12/backpack-show...,2005-06-12,1,6,572,412,4.035842,5.466019,0.01595,0.0,"[{'comment_ID': 570375, 'comment_content': 'Bu...","[{'inlink_ID': 319923, 'title': 'talking tech ..."
19461,19462,Plazes Profile,Michael Arrington,2,Company: PlazesWhat is it?Plazes made an annou...,http://techcrunch.com/2005/06/11/plazes-gets-t...,2005-06-11,10,12,292,217,4.313380,5.645161,0.29613,0.0,"[{'comment_ID': 570381, 'comment_content': 'Yo...","[{'inlink_ID': 319913, 'title': 'the time, the..."
19462,19463,FeedLounge Profile,Michael Arrington,2,Company: FeedLoungeWhat is it?The FeedLounge w...,http://techcrunch.com/2005/06/11/get-in-line-t...,2005-06-11,0,9,211,159,4.691176,6.018868,0.00000,0.0,"[{'comment_ID': 570393, 'comment_content': 'Yo...",[]


In [77]:
collection = database['posts']
collection.insert_many(posts.to_dict('records'))
collection = database['authors']
collection.insert_many(authors.to_dict('records'))